In [218]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "schools_complete.csv"
student_data_to_load = "students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)
school_data

,School ID,school_name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500
5,5,Wilson High School,Charter,2283,1319574
6,6,Cabrera High School,Charter,1858,1081356
7,7,Bailey High School,District,4976,3124928
8,8,Holden High School,Charter,427,248087
9,9,Pena High School,Charter,962,585858


In [185]:
# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [186]:
#Create list of schools and calculate number of schools
school_series = school_data_complete["school_name"].unique()
num_schools = round(len(school_series),0)

In [187]:
#Calculate total number of students
num_students = round(len(school_data_complete["student_name"]),0)
num_students

39170

In [188]:
#Calculate the total budget for district
budget_df = school_data_complete["budget"].unique()
total_budget = budget_df.sum()

In [189]:
#Calculate the average math and reading scores
math_av = round(school_data_complete["math_score"].mean(),2)
reading_av = round(school_data_complete["reading_score"].mean(),2)

In [190]:
#Calculate the overall average score
overall_av = round((math_av+reading_av)/2,2)

In [191]:
#Calculate % students with passing math score (70+)
mathpass_df = school_data_complete.loc[school_data_complete["math_score"]>=70]
mathpass_count = mathpass_df["math_score"].count()
mathpass_percent = round((mathpass_count/num_students*100),2)

In [192]:
#Calculate % students with passing reading score (70+)
readingpass_df = school_data_complete.loc[school_data_complete["reading_score"]>=70]
readingpass_count = readingpass_df["reading_score"].count()
readingpass_percent = round((readingpass_count/num_students*100),2)

In [193]:
#Create a DataFrame to hold the District Summary
district_df = pd.DataFrame({"Total Number of Schools": num_schools,"Total Number of Students":num_students, "Total Budget": total_budget, "Average Math Score": math_av, "Average Reading Score": reading_av, "% Overall Passing Rate": overall_av, "% Passing Math": mathpass_percent, "% Passing Reading": readingpass_percent}, index=[0])

district_df['Total Number of Students'] = district_df['Total Number of Students'].map("{:,.0f}".format)

district_df['Total Budget'] = district_df['Total Budget'].map("${:,.2f}".format)

district_df

,Total Number of Schools,Total Number of Students,Total Budget,Average Math Score,Average Reading Score,% Overall Passing Rate,% Passing Math,% Passing Reading
0,15,"39,170","$24,649,428.00",78.99,81.88,80.44,74.98,85.81


In [194]:
#Remove School ID column from school_data DF
school_data_no_id = school_data.iloc [:,1:]

In [195]:
#Change index to school name
school_data_new_index = school_data_no_id.set_index("school_name")
school_data_new_index

,type,size,budget
school_name,,,
Huang High School,District,2917,1910635
Figueroa High School,District,2949,1884411
Shelton High School,Charter,1761,1056600
Hernandez High School,District,4635,3022020
Griffin High School,Charter,1468,917500
Wilson High School,Charter,2283,1319574
Cabrera High School,Charter,1858,1081356
Bailey High School,District,4976,3124928
Holden High School,Charter,427,248087


In [196]:
#Rename columns and remove index label
school_data_renamed = school_data_new_index.rename(columns={"type": "School Type", "size":"Total Students","budget":"School Budget"})

school_data_renamed.index.name = None

In [197]:
#Create new column to calculate Per Student Budget
school_data_renamed["Per Student Budget"] = school_data_renamed["School Budget"]/school_data_renamed["Total Students"]

school_data_renamed["Per Student Budget"] = school_data_renamed["Per Student Budget"].map("{:,.2f}".format)

In [198]:
#Group data by school
grouped_data_by_school = school_data_complete.groupby('school_name')

#Add average math score as column to School Summary DF
school_data_renamed["Average Math Score"] = grouped_data_by_school['math_score'].mean()

school_data_renamed["Average Math Score"] = school_data_renamed["Average Math Score"].map("{:,.4f}".format)

In [199]:
#Calculate average reading score per school & add column to School Summary DF
school_data_renamed["Average Reading Score"] = grouped_data_by_school['reading_score'].mean()

school_data_renamed["Average Reading Score"] = school_data_renamed["Average Reading Score"].map("{:,.4f}".format)

In [200]:
#Calculate number of kids per school passing math
mathpass_by_school = mathpass_df["school_name"].value_counts()

#Sort series and DF to be used
mathpass_by_school.sort_index()
school_data_renamed.sort_index()

#Add % passing math as column to School Summary DF
school_data_renamed["% Passing Math"] =  (mathpass_by_school / school_data_renamed["Total Students"]*100)

school_data_renamed["% Passing Math"] = school_data_renamed["% Passing Math"].map("{:,.4f}".format)

In [201]:
#Calculate number of kids per school passing reading
readingpass_by_school = readingpass_df["school_name"].value_counts()

#Sort series and DF to be used
readingpass_by_school.sort_index()

#Add % passing reading as column to School Summary DF
school_data_renamed["% Passing Reading"] =  (readingpass_by_school / school_data_renamed["Total Students"]*100)

school_data_renamed["% Passing Reading"] = school_data_renamed["% Passing Reading"].map("{:,.4f}".format)

In [202]:
#Convert % passing math and reading columns from objects to numeric
school_data_renamed["% Passing Math"]=pd.to_numeric(school_data_renamed["% Passing Math"])
school_data_renamed["% Passing Reading"]=pd.to_numeric(school_data_renamed["% Passing Reading"])

#Add overall passing rate as column to School Summary DF
school_data_renamed["% Overall Passing Rate"] =  (school_data_renamed["% Passing Math"] + school_data_renamed["% Passing Reading"])/2

school_data_renamed["% Overall Passing Rate"] = school_data_renamed["% Overall Passing Rate"].map("{:,.4f}".format)
school_data_renamed

,School Type,Total Students,School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Huang High School,District,2917,1910635,655.00,76.6294,81.1827,65.6839,81.3164,73.5001
Figueroa High School,District,2949,1884411,639.00,76.7118,81.1580,65.9885,80.7392,73.3638
Shelton High School,Charter,1761,1056600,600.00,83.3595,83.7257,93.8671,95.8546,94.8608
Hernandez High School,District,4635,3022020,652.00,77.2898,80.9344,66.7530,80.8630,73.8080
Griffin High School,Charter,1468,917500,625.00,83.3515,83.8168,93.3924,97.1390,95.2657
Wilson High School,Charter,2283,1319574,578.00,83.2742,83.9895,93.8677,96.5396,95.2036
Cabrera High School,Charter,1858,1081356,582.00,83.0619,83.9758,94.1335,97.0398,95.5866
Bailey High School,District,4976,3124928,628.00,77.0484,81.0340,66.6801,81.9333,74.3067
Holden High School,Charter,427,248087,581.00,83.8033,83.8150,92.5059,96.2529,94.3794
Pena High School,Charter,962,585858,609.00,83.8399,84.0447,94.5946,95.9459,95.2703


## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [222]:
#Sort DF by passing rate
school_data_renamed_sorted = school_data_renamed.sort_values(["% Overall Passing Rate"], ascending=False)
school_data_renamed_sorted

,School Type,Total Students,School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Cabrera High School,Charter,1858,1081356,582.00,83.0619,83.9758,94.1335,97.0398,95.5866
Thomas High School,Charter,1635,1043130,638.00,83.4183,83.8489,93.2722,97.3089,95.2905
Pena High School,Charter,962,585858,609.00,83.8399,84.0447,94.5946,95.9459,95.2703
Griffin High School,Charter,1468,917500,625.00,83.3515,83.8168,93.3924,97.1390,95.2657
Wilson High School,Charter,2283,1319574,578.00,83.2742,83.9895,93.8677,96.5396,95.2036
Wright High School,Charter,1800,1049400,583.00,83.6822,83.9550,93.3333,96.6111,94.9722
Shelton High School,Charter,1761,1056600,600.00,83.3595,83.7257,93.8671,95.8546,94.8608
Holden High School,Charter,427,248087,581.00,83.8033,83.8150,92.5059,96.2529,94.3794
Bailey High School,District,4976,3124928,628.00,77.0484,81.0340,66.6801,81.9333,74.3067
Hernandez High School,District,4635,3022020,652.00,77.2898,80.9344,66.7530,80.8630,73.8080


In [204]:
#Display first 5 schools
school_data_renamed_sorted.iloc[0:5,:]

,School Type,Total Students,School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Cabrera High School,Charter,1858,1081356,582.00,83.0619,83.9758,94.1335,97.0398,95.5866
Thomas High School,Charter,1635,1043130,638.00,83.4183,83.8489,93.2722,97.3089,95.2905
Pena High School,Charter,962,585858,609.00,83.8399,84.0447,94.5946,95.9459,95.2703
Griffin High School,Charter,1468,917500,625.00,83.3515,83.8168,93.3924,97.1390,95.2657
Wilson High School,Charter,2283,1319574,578.00,83.2742,83.9895,93.8677,96.5396,95.2036


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [205]:
#Resort the DF and display bottom 5 schools
school_data_renamed_sorted = school_data_renamed.sort_values(["% Overall Passing Rate"], ascending=True)
school_data_renamed_sorted.iloc[0:5,:]

,School Type,Total Students,School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Rodriguez High School,District,3999,2547363,637.00,76.8427,80.7447,66.3666,80.2201,73.2934
Figueroa High School,District,2949,1884411,639.00,76.7118,81.1580,65.9885,80.7392,73.3638
Huang High School,District,2917,1910635,655.00,76.6294,81.1827,65.6839,81.3164,73.5001
Johnson High School,District,4761,3094650,650.00,77.0725,80.9664,66.0576,81.2224,73.6400
Ford High School,District,2739,1763916,644.00,77.1026,80.7463,68.3096,79.2990,73.8043


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [206]:
#Create series for each grade's data
scores_9th = school_data_complete.loc[school_data_complete['grade']=="9th",:]
scores_10th = school_data_complete.loc[school_data_complete['grade']=="10th",:]
scores_11th = school_data_complete.loc[school_data_complete['grade']=="11th",:]
scores_12th = school_data_complete.loc[school_data_complete['grade']=="12th",:]

In [207]:
#Calculate average math scores by school for 9th grade
math_9th_group = scores_9th.groupby(["school_name"])
math_9th_av = math_9th_group["math_score"].mean()

#Create dataframe for average math scores by school and grade
math_by_grade = pd.DataFrame(math_9th_av)

#Remove index label and rename column
math_by_grade.index.name = None
math_by_grade = math_by_grade.rename(columns = {"math_score":"9th"})

In [208]:
#Calculate and append the rest of the average grades to the DF
math_10th_group = scores_10th.groupby(["school_name"])
math_10th_av = math_10th_group["math_score"].mean()
math_by_grade["10th"] = math_10th_av

math_11th_group = scores_11th.groupby(["school_name"])
math_11th_av = math_11th_group["math_score"].mean()
math_by_grade["11th"] = math_11th_av

math_12th_group = scores_12th.groupby(["school_name"])
math_12th_av = math_12th_group["math_score"].mean()
math_by_grade["12th"] = math_12th_av

math_by_grade

,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


## Reading Score by Grade 

* Perform the same operations as above for reading scores

,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


In [209]:
#Calculate average reading scores by school for 9th grade
reading_9th_group = scores_9th.groupby(["school_name"])
reading_9th_av = reading_9th_group["reading_score"].mean()

#Create dataframe for average reading scores by school and grade
reading_by_grade = pd.DataFrame(reading_9th_av)

#Remove index label and rename column
reading_by_grade.index.name = None
reading_by_grade = reading_by_grade.rename(columns = {"reading_score":"9th"})

In [210]:
#Calculate and append the rest of the average grades to the DF
reading_10th_group = scores_10th.groupby(["school_name"])
reading_10th_av = reading_10th_group["reading_score"].mean()
reading_by_grade["10th"] = reading_10th_av

reading_11th_group = scores_11th.groupby(["school_name"])
reading_11th_av = reading_11th_group["reading_score"].mean()
reading_by_grade["11th"] = reading_11th_av

reading_12th_group = scores_12th.groupby(["school_name"])
reading_12th_av = reading_12th_group["reading_score"].mean()
reading_by_grade["12th"] = reading_12th_av

reading_by_grade

,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [219]:
#Use df with unaveraged score data (school_data_complete)
#Add budget per student as column
school_spending_df = school_data_complete
school_spending_df

#Calculate number of students based on school and add as row
school_spending_df["num_students"] = school_spending_df["budget"]/

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635
...,...,...,...,...,...,...,...,...,...,...,...
39165,39165,Donna Howard,F,12th,Thomas High School,99,90,14,Charter,1635,1043130
39166,39166,Dawn Bell,F,10th,Thomas High School,95,70,14,Charter,1635,1043130
39167,39167,Rebecca Tanner,F,9th,Thomas High School,73,84,14,Charter,1635,1043130
39168,39168,Desiree Kidd,F,10th,Thomas High School,99,90,14,Charter,1635,1043130


In [220]:
# Create bins and labels based on sample bins
spending_bins = [555, 585, 615, 645, 675]
group_names = ["$555-584", "$585-614", "$615-644", "$645-674"]

In [223]:
#Slice data with bins for budget per student
school_spending_df["Spending Bins"] = pd.cut(school_data_complete['Per Student Budget'],spending_bins,labels=group_names)

KeyError: 'Per Student Budget'

In [ ]:
#Create a groupby based on the spending bins
#spending_bins = school_data_renamed.groupby("Spending Bins")
#spending_bins_df = spending_bins.get_group("$555-584")

#spending_bins[["Per Student Budget","Average Math Score","Average Reading Score","% Passing Math","% Passing Reading","% Overall Passing Rate"]]

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,95.035486
$585-615,83.599686,83.885211,94.230858,95.900287,95.065572
$615-645,79.079225,81.891436,75.668212,86.106569,80.887391
$645-675,76.997210,81.027843,66.164813,81.133951,73.649382


## Scores by School Size

* Perform the same operations as above, based on school size.

In [224]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
size_labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [225]:
#Slice data with bins for budget per student
school_spending_df["School Size"] = pd.cut(school_data_complete['size'],size_bins,labels=size_labels)
school_spending_df.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget,School Size
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635,Large (2000-5000)
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635,Large (2000-5000)
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635,Large (2000-5000)
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635,Large (2000-5000)
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635,Large (2000-5000)


In [231]:
#Create a groupby based on the school size bins and calculate mean score per size bin
scores_by_size = school_spending_df.groupby("School Size")
scores_by_size_mean = scores_by_size.mean()
scores_size_formatted = scores_by_size_mean[["math_score","reading_score"]]
scores_size_formatted = scores_size_formatted.rename(columns = {"math_score":"Average Math Score","reading_score":"Average Reading Score"})
scores_size_formatted

,Average Math Score,Average Reading Score
School Size,,
Small (<1000),83.828654,83.974082
Medium (1000-2000),83.372682,83.867989
Large (2000-5000),77.477597,81.198674


In [232]:
#Calculate number of students in each size bin
scores_by_size_count = scores_by_size.count()
scores_by_size_count

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
School Size,,,,,,,,,,,
Small (<1000),1389,1389,1389,1389,1389,1389,1389,1389,1389,1389,1389
Medium (1000-2000),8522,8522,8522,8522,8522,8522,8522,8522,8522,8522,8522
Large (2000-5000),29259,29259,29259,29259,29259,29259,29259,29259,29259,29259,29259


In [244]:
#Create DF for only students passing math
scores_by_size_mathpass = school_spending_df.loc[(school_spending_df["math_score"])>69,:]

#Create a groupby based on the school size bins and count number of students in df per size bin
scores_by_size_mathpass_group = scores_by_size_mathpass.groupby("School Size")
scores_by_size_mathpass_count = scores_by_size_mathpass_group.count()
scores_by_size_mathpass_count

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
School Size,,,,,,,,,,,
Small (<1000),1305,1305,1305,1305,1305,1305,1305,1305,1305,1305,1305
Medium (1000-2000),7978,7978,7978,7978,7978,7978,7978,7978,7978,7978,7978
Large (2000-5000),20087,20087,20087,20087,20087,20087,20087,20087,20087,20087,20087


In [246]:
#Add new column for % passing math
scores_size_formatted["% Passing Math"] = scores_by_size_mathpass_count["gender"]/scores_by_size_count["gender"]*100
scores_size_formatted

,Average Math Score,Average Reading Score,% Passing Math
School Size,,,
Small (<1000),83.828654,83.974082,93.952484
Medium (1000-2000),83.372682,83.867989,93.616522
Large (2000-5000),77.477597,81.198674,68.652380


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,94.824831
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,76.364998


## Scores by School Type

* Perform the same operations as above, based on school type.

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757
